In [ ]:
#!pip install mysql-connector-python

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import warnings
import getpass
import mysql.connector as msql
from mysql.connector import Error
warnings.filterwarnings('ignore')

Import CSV Files

In [2]:
calendar=pd.read_csv('Clean/calendar.csv',index_col = False, delimiter = ',')
reviews=pd.read_csv('Clean/reviews.csv',index_col = False, delimiter = ',')
listings=pd.read_csv('Clean/listings.csv',index_col = False, delimiter = ',')

In [3]:
print(f'calendar dataframe shape {calendar.shape}')
print(f'listings dataframe shape {listings.shape}')
print(f'reviews dataframe shape {reviews.shape}')

calendar dataframe shape (1048575, 4)
listings dataframe shape (3818, 95)
reviews dataframe shape (84849, 8)


Connect to MySQL

In [4]:
def connect_to_msql(database=''):
    try:
        conn = msql.connect(host='localhost', user='root', password=getpass.getpass('Enter password:'), database=f'{database}')
        print("Connected to MySQL...")
    except Error as e:
        print("Error while connecting to MySQL", e)
    return conn

# Run initial DB connection
conn = connect_to_msql()


Connected to MySQL...


Drop Database

In [5]:
# Function for dropping DB

def drop_database(connection, string):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
        else: 
            connection = connect_to_msql()
            cursor = connection.cursor()

        cursor.execute(f"DROP DATABASE IF EXISTS {string}")
        print(f"Database: {string} is dropped")
    except Error as e:
        print("Error while connecting to MySQL", e)

# How to call function, with conn and DB string

drop_database(conn, 'Airbnb_DB')

Database: Airbnb_DB is dropped


Create Database

In [6]:
# Function for creating DB

def create_database(connection, string):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
        else:
            connection = connect_to_msql()
            cursor = connection.cursor()
        cursor.execute(f"CREATE DATABASE {string}")
        print(f"Database: {string} has been created...")
    except Error as e:
        print(f"Error while creating database: {string}", e)

# How to call function, with conn and DB string

create_database(conn, 'Airbnb_DB')

Database: Airbnb_DB has been created...


In [7]:
# Function to disable foreign key checks

def disable_fk_checks(connection):
    cursor = connection.cursor()
    cursor.execute("""
        SET FOREIGN_KEY_CHECKS = 0;
        SET GLOBAL FOREIGN_KEY_CHECKS = 0;
    """, multi=True)
    connection.commit()
    print("Foreign Key Checks Disabled...")

# Function to enable foreign key checks

def enable_fk_checks(connection):
    cursor = connection.cursor()
    cursor.execute("""
        SET FOREIGN_KEY_CHECKS = 1;
        SET GLOBAL FOREIGN_KEY_CHECKS = 1;
    """, multi=True)
    connection.commit()
    print("Foreign Key Checks Enabled...")

# Function to find unique array values

def unique(array):
    x = np.array(array)
    print("Unique list values: ", np.unique(x))
    return np.unique(x)

# Call disable function with conn

disable_fk_checks(conn)

Foreign Key Checks Disabled...


In [8]:
listings

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,summary_sentiment,space_sentiment,description_sentiment
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,WASHINGTON,0,moderate,0,0,2,4.07,0.000000,0.182917,0.182917
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,WASHINGTON,0,strict,1,1,6,1.48,0.183333,0.397868,0.348630
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,WASHINGTON,0,strict,0,0,2,1.15,0.392338,0.205905,0.273670
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,WASHINGTON,0,flexible,0,0,1,NaN,0.675000,0.000000,0.675000
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,WASHINGTON,0,strict,0,0,1,0.89,0.550000,0.288265,0.344451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,8101950,https://www.airbnb.com/rooms/8101950,20160104002432,2016-01-04,3BR Mountain View House in Seattle,Our 3BR/2BA house boasts incredible views of t...,"Our 3BR/2BA house bright, stylish, and wheelch...",Our 3BR/2BA house boasts incredible views of t...,none,We're located near lots of family fun. Woodlan...,...,WASHINGTON,0,strict,0,0,8,0.30,0.575000,0.383664,0.531111
3814,8902327,https://www.airbnb.com/rooms/8902327,20160104002432,2016-01-04,Portage Bay View!-One Bedroom Apt,800 square foot 1 bedroom basement apartment w...,This space has a great view of Portage Bay wit...,800 square foot 1 bedroom basement apartment w...,none,The neighborhood is a quiet oasis that is clos...,...,WASHINGTON,0,moderate,0,0,1,2.00,0.228571,0.500000,0.215164
3815,10267360,https://www.airbnb.com/rooms/10267360,20160104002432,2016-01-04,Private apartment view of Lake WA,"Very comfortable lower unit. Quiet, charming m...",NaN,"Very comfortable lower unit. Quiet, charming m...",none,NaN,...,WASHINGTON,0,moderate,0,0,1,NaN,0.450667,0.000000,0.450667
3816,9604740,https://www.airbnb.com/rooms/9604740,20160104002432,2016-01-04,Amazing View with Modern Comfort!,Cozy studio condo in the heart on Madison Park...,Fully furnished unit to accommodate most needs...,Cozy studio condo in the heart on Madison Park...,none,Madison Park offers a peaceful slow pace upsca...,...,WASHINGTON,0,moderate,0,0,1,NaN,0.258333,0.296667,0.287670


Import Listings CSV File

Import ETA: 10 seconds

In [11]:
# Function for importing listings file

listings = listings.replace(np.nan, None)

def import_listings(connection, string):

    try:                  
        if connection.is_connected():
            cursor = connection.cursor()
        else:
            connection = connect_to_msql(f'{string}')
            cursor = connection.cursor()
        cursor.execute(f"USE {string};")
        cursor.execute("SELECT database();")

        record = cursor.fetchone()
        print("You're connected to database: ", record)

        cursor.execute('DROP TABLE IF EXISTS listings;')

        print('Creating Listings table....')

        cursor.execute("""
        CREATE TABLE listings( 
            id INT UNSIGNED NOT NULL,
            listing_url TEXT DEFAULT NULL,
            scrape_id BIGINT DEFAULT NULL,
            last_scraped DATE DEFAULT NULL,
            name VARCHAR(100) DEFAULT NULL,
            summary TEXT DEFAULT NULL,
            space TEXT DEFAULT NULL,
            description TEXT DEFAULT NULL,
            experiences_offered VARCHAR(10) DEFAULT NULL,
            neighborhood_overview TEXT DEFAULT NULL,
            notes TEXT DEFAULT NULL,
            transit TEXT DEFAULT NULL,
            thumbnail_url TEXT DEFAULT NULL,
            medium_url TEXT DEFAULT NULL,
            picture_url TEXT DEFAULT NULL,
            xl_picture_url TEXT DEFAULT NULL,
            host_id INT UNSIGNED DEFAULT NULL,
            host_url TEXT DEFAULT NULL,
            host_name VARCHAR (100) DEFAULT NULL,
            host_since DATE DEFAULT NULL,
            host_location VARCHAR (100) DEFAULT NULL,
            host_about TEXT DEFAULT NULL,
            host_response_time VARCHAR(100) DEFAULT NULL,
            host_response_rate VARCHAR(10) DEFAULT NULL,
            host_acceptance_rate VARCHAR(10) DEFAULT NULL,
            host_is_superhost BOOLEAN DEFAULT NULL,
            host_thumbnail_url TEXT DEFAULT NULL,
            host_picture_url TEXT DEFAULT NULL,
            host_neighbourhood VARCHAR (100) DEFAULT NULL,
            host_listings_count INT DEFAULT NULL,
            host_total_listings_count INT DEFAULT NULL,
            host_verifications VARCHAR (500) DEFAULT NULL,
            host_has_profile_pic BOOLEAN DEFAULT NULL,
            host_identity_verified BOOLEAN DEFAULT NULL,
            street TEXT DEFAULT NULL,
            neighbourhood VARCHAR (100) DEFAULT NULL,
            neighbourhood_cleansed VARCHAR (100) DEFAULT NULL,
            neighbourhood_group_cleansed VARCHAR (100) DEFAULT NULL,
            city VARCHAR (100) DEFAULT NULL,
            state VARCHAR (10) DEFAULT NULL,
            zipcode INT DEFAULT NULL,
            market VARCHAR(50) DEFAULT NULL,
            smart_location VARCHAR (25) DEFAULT NULL,
            country_code TEXT,
            country VARCHAR(50) DEFAULT NULL,
            latitude VARCHAR(50) DEFAULT NULL,
            longitude VARCHAR(50) DEFAULT NULL,
            is_location_exact BOOLEAN DEFAULT NULL,
            property_type VARCHAR(50) DEFAULT NULL,
            room_type VARCHAR(50) DEFAULT NULL,
            accommodates SMALLINT UNSIGNED NULL,
            bathrooms FLOAT UNSIGNED DEFAULT NULL,
            bedrooms SMALLINT UNSIGNED DEFAULT NULL,
            beds SMALLINT UNSIGNED DEFAULT NULL,
            bed_type VARCHAR(50) DEFAULT NULL,
            amenities TEXT DEFAULT NULL,
            square_feet BIGINT DEFAULT NULL,
            price VARCHAR(10) DEFAULT NULL,
            weekly_price VARCHAR(10) DEFAULT NULL,
            monthly_price VARCHAR(10) DEFAULT NULL,
            security_deposit VARCHAR(10) DEFAULT NULL,
            cleaning_fee VARCHAR(10) DEFAULT NULL,
            guests_included SMALLINT UNSIGNED DEFAULT NULL,
            extra_people VARCHAR(10) DEFAULT NULL,
            minimum_nights SMALLINT UNSIGNED DEFAULT NULL,
            maximum_nights INT UNSIGNED DEFAULT NULL,
            calendar_updated VARCHAR(50) DEFAULT NULL,
            has_availability BOOLEAN DEFAULT NULL,
            availability_30 SMALLINT UNSIGNED NOT NULL,
            availability_60 SMALLINT UNSIGNED NOT NULL,
            availability_90 SMALLINT UNSIGNED NOT NULL,
            availability_365 SMALLINT UNSIGNED NOT NULL,
            calendar_last_scraped DATE DEFAULT NULL,
            number_of_reviews SMALLINT UNSIGNED NULL,
            first_review DATE DEFAULT NULL,
            last_review DATE DEFAULT NULL,
            review_scores_rating SMALLINT UNSIGNED DEFAULT NULL,
            review_scores_accuracy SMALLINT UNSIGNED DEFAULT NULL,
            review_scores_cleanliness SMALLINT UNSIGNED DEFAULT NULL,
            review_scores_checkin SMALLINT UNSIGNED DEFAULT NULL,
            review_scores_communication SMALLINT UNSIGNED NULL DEFAULT NULL,
            review_scores_location SMALLINT UNSIGNED DEFAULT NULL,
            review_scores_value SMALLINT UNSIGNED DEFAULT NULL,
            requires_license BOOLEAN DEFAULT NULL,
            license VARCHAR(50) NULL DEFAULT NULL,
            jurisdiction_names VARCHAR(50) DEFAULT NULL,
            instant_bookable BOOLEAN DEFAULT NULL,
            cancellation_policy VARCHAR(50) NULL DEFAULT NULL,
            require_guest_profile_picture BOOLEAN DEFAULT NULL,
            require_guest_phone_verification BOOLEAN DEFAULT NULL,
            calculated_host_listings_count SMALLINT UNSIGNED NULL DEFAULT NULL,
            reviews_per_month FLOAT UNSIGNED NULL,
            summary_sentiment FLOAT NOT NULL,
            space_sentiment FLOAT NOT NULL,
            description_sentiment FLOAT NOT NULL,
            PRIMARY KEY (id)
        )
        """)
    
        print("Listings table has been created....")
        
         #loop through the data frame
        for i,row in listings.iterrows():
            try:
                #here %S means string values 
                sql = """
                INSERT INTO Airbnb_DB.listings VALUES (
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                    %s,%s,%s,%s,%s)
                """
                
                cursor.execute(sql, tuple(row))
                # print("Record inserted")
                
                connection.commit()
            except Error as e:
                print("Error while importing Listings: ", e)
                
        print("Number of records inserted into listings table: ", i + 1)
    except Error as e:
        print("Error while connecting to MySQL", e)

# How to call function

import_listings(conn, 'Airbnb_DB')


You're connected to database:  ('airbnb_db',)
Creating Listings table....
Listings table has been created....
Number of records inserted into listings table:  3818


In [13]:
# Run initial DB connection

conn = connect_to_msql('Airbnb_DB')

# Lambda function for queries

pysqldf = lambda q: pd.read_sql(q, conn)

Connected to MySQL...


In [14]:
q = """
SELECT * FROM listings;
"""

res = pysqldf(q)

res.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,summary_sentiment,space_sentiment,description_sentiment
0,3335,https://www.airbnb.com/rooms/3335,20160104002432,2016-01-04,Sweet Seattle Urban Homestead 2 Bdr,Welcome! If you stay here you will be living i...,Welcome! Come enjoy your time in Seattle at a...,Welcome! If you stay here you will be living i...,none,This is a very diverse neighborhood with a var...,...,WASHINGTON,0,strict,0,0,4,NaN,0.495833,0.275758,0.359137
1,4291,https://www.airbnb.com/rooms/4291,20160104002432,2016-01-04,Sunrise in Seattle Master Suite,None,"Located in Seattle, this is a spacious, clean...","Located in Seattle, this is a spacious, clean...",none,None,...,WASHINGTON,0,moderate,0,0,5,1.14,0.000000,0.178011,0.178011
2,5682,https://www.airbnb.com/rooms/5682,20160104002432,2016-01-04,"Cozy Studio, min. to downtown -WiFi",The Cozy Studio is a perfect launchpad for you...,"Hello fellow travelers, Save some money and ha...",The Cozy Studio is a perfect launchpad for you...,none,None,...,WASHINGTON,1,strict,0,1,1,4.21,0.366667,0.204940,0.274513
3,6606,https://www.airbnb.com/rooms/6606,20160104002432,2016-01-04,"Fab, private seattle urban cottage!",None,"Soo centrally located, this is a little house ...","Soo centrally located, this is a little house ...",none,"A peaceful yet highly accessible neighborhood,...",...,WASHINGTON,0,strict,0,0,1,0.66,0.000000,0.162820,0.162820
4,7369,https://www.airbnb.com/rooms/7369,20160104002432,2016-01-04,launchingpad/landingpad,contemporary condo on the western edge of pike...,spacious condo with all the amenities,contemporary condo on the western edge of pike...,none,Pike/Pine is the most urban neighborhood in se...,...,WASHINGTON,0,flexible,0,0,1,0.50,0.141667,0.000000,0.250918


Import Reviews CSV

Import ETA: 30 seconds

In [15]:
reviews = reviews.replace(np.nan, None)

# Function for importing reviews files

def import_reviews(connection, string):

    try:                  
        if connection.is_connected():
            cursor = connection.cursor()
        else:
            connection = connect_to_msql(f'{string}')
            cursor = connection.cursor()
        cursor.execute(f"USE {string};")
        cursor.execute("SELECT database();")

        record = cursor.fetchone()
        print("You're connected to database: ", record)

        cursor.execute('DROP TABLE IF EXISTS reviews;')

        print('Creating Reviews table....')

        # pass the create table statement
        cursor.execute("""
        CREATE TABLE reviews(
            listing_id int UNSIGNED NOT NULL,
            review_id int,
            date date,
            reviewer_id int DEFAULT NULL,
            reviewer_name varchar(100) DEFAULT NULL, 
            comments text DEFAULT NULL,
            sentiment FLOAT DEFAULT NULL,
            sentiment_label VARCHAR(10) DEFAULT NULL,
            PRIMARY KEY (review_id),
            FOREIGN KEY (listing_id)
                REFERENCES listings (id) 
                ON DELETE RESTRICT 
                ON UPDATE CASCADE
        )
        """)
        print("Reviews table has been created....")
        
        nonexistent_listings = []
        for i,row in reviews.iterrows():
            try:
                sql = "INSERT INTO Airbnb_DB.reviews VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql, tuple(row))
                # print("Record inserted")
                
                connection.commit()
            except Error as e:
                print("Error while importing Reviews: ", e)
                nonexistent_listings.append(tuple(row)[0])

        unique(nonexistent_listings)
        print("\nNumber of records inserted into reviews table: ", i + 1 - len(nonexistent_listings))
    except Error as e:
        print("Error while connecting to MySQL", e)

# How to call function

import_reviews(conn, 'Airbnb_DB')

You're connected to database:  ('airbnb_db',)
Creating Reviews table....
Reviews table has been created....
Unique list values:  []

Number of records inserted into reviews table:  84849


Import Calendar CSV Files

Import ETA: 4 - 5 mins

In [16]:
# Function for importing calendar files

calendar = calendar.replace(np.nan, None)

def import_calendar(connection, string):

    try:                  
        if connection.is_connected():
            cursor = connection.cursor()
        else:
            connection = connect_to_msql(f'{string}')
            cursor = connection.cursor()
        cursor.execute(f"USE {string};")
        cursor.execute("SELECT database();")

        record = cursor.fetchone()
        print("You're connected to database: ", record)

        cursor.execute('DROP TABLE IF EXISTS calendar;')

        print('Creating Calendar table....')
                        
        # pass the create table statement 
        cursor.execute("""
        CREATE TABLE calendar( 
            calendar_id int NOT NULL AUTO_INCREMENT,
            listing_id INT UNSIGNED NOT NULL,
            date DATE DEFAULT NULL,
            available TEXT DEFAULT NULL, 
            price FLOAT DEFAULT NULL,
            PRIMARY KEY (calendar_id),
            FOREIGN KEY (listing_id) 
                REFERENCES listings (id) 
                ON DELETE RESTRICT 
                ON UPDATE CASCADE
            );
        """)

        print("Calendar table has been created....")

        nonexistent_listings = []
        for i,row in calendar.iterrows():
            try:
                sql = "INSERT INTO Airbnb_DB.calendar(listing_id, date, available, price) VALUES (%s,%s,%s,%s)" # %S means string interpolation
                cursor.execute(sql, tuple(row))
                connection.commit() # since connection is not auto committed by default
            except Error as e:
                print("Error while importing Calendar: ", e)
                nonexistent_listings.append(tuple(row)[0])

        unique(nonexistent_listings)
        print("Number of records inserted into calendar table: ", i + 1 - len(nonexistent_listings))
    except Error as e:
        print("Error while connecting to MySQL", e)

# How to call function

import_calendar(conn, 'Airbnb_DB')


You're connected to database:  ('airbnb_db',)
Creating Calendar table....
Calendar table has been created....
Unique list values:  []
Number of records inserted into calendar table:  1048575
